In [1]:
import pandas as pd
import urllib

import numpy as np

import json

from tqdm.autonotebook import tqdm

#%matplotlib inline

tqdm.pandas()

import dask.dataframe as dd

from dask.multiprocessing import get
from dask.diagnostics import ProgressBar

from datetime import datetime
import matplotlib.pyplot as plt

from IPython.display import display


/home/vandy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [2]:
import urllib3

In [3]:
http = urllib3.PoolManager()

In [4]:
from config_batch import * 

# Functions

In [5]:
ws_hostname = "127.0.1.1"
# ws_hostname = "192.168.1.3"


In [6]:
def call_ws(addr_data, check_result=True, structured_osm=False): #lg = "en,fr,nl"
    t = datetime.now()
    
    params = urllib.parse.urlencode({"street": addr_data[street_field],
                                     "housenumber": addr_data[housenbr_field],
                                     "city": addr_data[city_field],
                                     "postcode": addr_data[postcode_field],
                                     "country": addr_data[country_field],
                                     "check_result" : "yes" if check_result else "no",
                                     "struct_osm" : "yes" if structured_osm else "no"
                                    })
    url = f"http://{ws_hostname}:5000/search/?{params}"
    
    try:
        with urllib.request.urlopen(url) as response:
            res = response.read()
            res = json.loads(res)
#             print(res)
            res["time"] = datetime.now() - t
            return res
    except Exception as e:
        return str(e)
    

In [7]:
def call_ws_batch(addr_data, mode="geo", with_reject=False, check_result=True, structured_osm=False): #lg = "en,fr,nl"
#     print(addr_data)
#     print(addr_data.shape)
#     print()
    file_data = addr_data.rename(columns = {
        street_field : "street",
        housenbr_field: "housenumber",
        postcode_field: "postcode",
        city_field: "city",
        country_field: "country",
        addr_key_field : "addr_key"
    }).to_csv(index=False)
    
    r = http.request(
    'POST',
    f'http://{ws_hostname}:5000/batch',
    fields= { 
        'media': ('addresses.csv', file_data),
        'mode': mode,
        "with_rejected" : "yes" if with_reject else "no",
        "check_result" : "yes" if check_result else "no",
        "struct_osm" : "yes" if structured_osm else "no"
    })
    
    try:
        res = pd.DataFrame(json.loads(r.data.decode('utf-8')))
    except ValueError:
        print("Cannot decode result:")
        print(json.loads(r.data.decode('utf-8')))
        return 
#     display(res)
    return res

In [8]:
def expand_json(addresses):
    addresses["status"]= addresses.json.apply(lambda d: "error" if "error" in d else "match" if "match" in d else "rejected")
    addresses["time"]  = addresses.json.apply(lambda d: d["time"])

    addresses["timing"]  = addresses.json.apply(lambda d: d["timing"] if "timing" in d else {})

    addresses["method"]= addresses.json.apply(lambda d: d["match"][0]["method"] if len(d)>0 and "match" in d else "none")
    
    for field in ["street", "number", "postcode", "city"]:
        addresses[field]= addresses.json.apply(lambda d: d["match"][0]["addr_out_"+field] if len(d)>0 and "match" in d else "")
    return 

# Calls

## Single address calls

In [11]:
call_ws({street_field:   "Av. Fonsny", 
         housenbr_field: "20",
         city_field:     "Saint-Gilles",
         postcode_field: "1060",
         country_field:  "Belgium"}, check_result=False, structured_osm=False)

{'match': [{'SIM_street_which': '',
   'addr_out_city': 'Saint-Gilles - Sint-Gillis',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '20',
   'addr_out_postcode': '',
   'addr_out_street': 'Avenue Fonsny - Fonsnylaan',
   'display_name': 'DAE (Smals), 20, Avenue Fonsny - Fonsnylaan, Saint-Gilles - Sint-Gillis, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, België / Belgique / Belgien',
   'extra_house_nbr': '20',
   'lat': '50.8358216',
   'lon': '4.3386884',
   'method': 'orig',
   'osm_addr_in': 'Av. Fonsny, 20, 1060 Saint-Gilles, Belgium',
   'place_id': 343087,
   'place_rank': 30}],
 'rejected': [{'SIM_street': '',
   'SIM_zip': '',
   'addr_out_city': 'Saint-Gilles - Sint-Gillis',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '20',
   'addr_out_postcode': '',
   'addr_out_street': 'Avenue Fonsny - Fonsnylaan',
   'display_name': '20, Avenue Fonsny - Fonsnylaan, Saint

In [160]:
call_ws({street_field:   "Avenue Louise", 
         housenbr_field: "375",
         city_field:     "Ixelles",
         postcode_field: "1050",
         country_field:  "Belgium"}, check_result=True, structured_osm=True)

{'match': [{'SIM_city': 1.0,
   'SIM_house_nbr': 0.0,
   'SIM_street': 1.0,
   'SIM_street_which': 'addr_out_street',
   'SIM_zip': 1.0,
   'addr_out_city': 'Ixelles - Elsene',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '',
   'addr_out_postcode': '1050',
   'addr_out_street': 'Avenue Louis Lepoutre - Louis Lepoutrelaan',
   'display_name': 'Avenue Louis Lepoutre - Louis Lepoutrelaan, Ixelles - Elsene, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1050, België / Belgique / Belgien',
   'extra_house_nbr': '375',
   'lat': '50.8196702',
   'lon': '4.3586694',
   'method': 'libpostal+regex[lpost]+photon',
   'osm_addr_in': 'Avenue Louis Lepoutre - Louis Lepoutrelaan, 375, 1050 Ixelles - Elsene, België - Belgique - Belgien',
   'place_id': 521112,
   'place_rank': 26}],
 'rejected': [],
 'time': datetime.timedelta(seconds=2, microseconds=789996)}

## Batch calls (row by row)

In [15]:
addresses = get_addresses("address.csv.gz")
addresses = addresses.sample(1000).copy()

### Simple way

In [74]:
addresses["json"] = addresses.progress_apply(call_ws, check_result=True, structured_osm=False, axis=1)

  0%|          | 0/100 [00:00<?, ?it/s]

### Using Dask

In [17]:
dd_addresses = dd.from_pandas(addresses, npartitions=4)

dask_task = dd_addresses.apply(call_ws, meta=('x', 'str'), axis=1)

with ProgressBar(): 
    addresses["json"] = dask_task.compute()

[########################################] | 100% Completed | 59.1s


In [26]:
expand_json(addresses)

In [27]:
addresses

,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber,json,status,time,timing,method,street,number,postcode,city
1321156,0836.380.520,Belgique,7911,Frasnes-lez-Anvaing,Rue Goderneau(MB),14,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.960761,{},regex[init],Rue Goderneau,,7906,Ath
2660061,2.231.595.460,Belgique,9981,Sint-Laureins,Vlamingstraat(STM),92,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:01.143796,{},regex[init],Vlamingstraat,,9981,Sint-Laureins
164680,0431.356.723,Belgique,1050,Ixelles,Rue de l'Amazone,21,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.724923,{},orig,Rue de l'Amazone - Amazonestraat,21,1050,Ixelles - Elsene
1295595,0832.380.556,Belgique,4000,Liège,Rue Henri-Maus,29,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.584219,{},orig,Rue Henri Maus,,4000,Liège
2293307,2.168.200.715,Belgique,8300,Knokke-Heist,Zeedijk-Knokke,641,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.548851,{},orig,Zeedijk Knokke,,8300,Knokke-Heist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2751112,2.244.614.642,Belgique,9240,Zele,Smeiersberg,5,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.544249,{},orig,Smeiersberg,,9240,Zele
2810314,2.252.606.947,Belgique,1080,Molenbeek-Saint-Jean,Rue Henri De Saegher,27,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.968186,{},orig,Rue Henri De Saegher - Henri De Saegherstraat,27,1080,Molenbeek-Saint-Jean - Sint-Jans-Molenbeek
732484,0650.498.331,Belgique,4700,Eupen,Stockbergerweg,5,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.987093,{},orig,Stockbergerweg,,4700,Eupen
1799145,2.037.557.947,Belgique,1180,Uccle,Parvis Saint-Pierre,119,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.517703,{},orig,Parvis Saint-Pierre - Sint-Pietersvoorplein,,1180,Uccle - Ukkel


## Batch calls (batch WS)

### Single block

In [53]:
# Only geocoding
call_ws_batch(addresses, mode="geo", check_result=True)

,addr_key,lat,lon,place_rank,method
0,0308.251.647,50.467047,4.207648,26.0,orig
1,0408.704.352,50.894373,4.379952,30.0,orig
2,0413.358.372,51.220346,4.413114,30.0,orig
3,0431.356.723,50.824780,4.359237,30.0,orig
4,0439.768.306,50.190684,4.535742,26.0,orig
...,...,...,...,...,...
93,0453.040.676,50.270696,5.521461,26.0,nonum
94,0535.902.531,50.292862,5.031069,30.0,libpostal+regex[lpost]+photon
95,2.202.472.003,50.878578,4.409547,26.0,libpostal+regex[lpost]+photon
96,2.237.979.050,50.590356,5.747109,26.0,libpostal+regex[lpost]+photon


In [62]:
# Geocode + address
call_ws_batch(addresses, mode="short") 

,addr_key,lat,lon,place_rank,method,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country
0,2.164.397.226,50.712641,4.530106,26.0,orig,Rue Robert Boisacq,None,9 A,1330,Rixensart,België / Belgique / Belgien
1,2.279.599.374,50.859702,2.828440,26.0,orig,Stijn Streuvelswijk,None,18,8908,Ieper,België / Belgique / Belgien
2,0669.833.203,51.209175,4.470072,30.0,orig,Boterlaarbaan,239,239,2100,Deurne,België / Belgique / Belgien
3,2.243.108.073,50.658359,5.483832,26.0,orig,Rue de l'Aéropostale,None,8,4460,Grâce-Hollogne,België / Belgique / Belgien
4,2.045.477.107,51.100407,3.167726,26.0,orig,Koningin Astridstraat,None,109,8210,Zedelgem,België / Belgique / Belgien
...,...,...,...,...,...,...,...,...,...,...,...
92,0857.952.627,50.834163,3.231044,26.0,regex[init],Heulsestraat,None,3,8501,Kortrijk,België / Belgique / Belgien
93,2.249.583.121,51.194730,4.407469,30.0,orig,Desguinlei,88-90,90,2018,Antwerpen,België / Belgique / Belgien
94,0834.315.707,51.204519,4.398727,30.0,orig,Diercxsensstraat,39,39,2018,Antwerpen,België / Belgique / Belgien
95,0857.635.002,50.843955,4.264415,26.0,orig,Ninoofsesteenweg,None,227,1700,Dilbeek,België / Belgique / Belgien


In [63]:
# Geocode + address, with rejected addresses
call_ws_batch(addresses, mode="long", with_reject=True) 

,addr_key,country,postcode,city,street,housenumber,osm_addr_in,place_id,lat,lon,...,addr_out_street,addr_out_city,addr_out_number,addr_out_country,addr_out_postcode,addr_out_other,retry_on_26,method,extra_house_nbr,rejected
0,2.164.397.226,Belgique,1330,Rixensart,Rue Robert Boisacq,9 A,"Rue Robert Boisacq, 9 A, 1330 Rixensart, Belgique",564275.0,50.712641,4.530106,...,Rue Robert Boisacq,Rixensart,None,België / Belgique / Belgien,1330,Fond du Patch,None,orig,9 A,None
1,2.279.599.374,Belgique,8908,Ieper,Streuvelswijk,18,"Streuvelswijk, 18, 8908 Ieper, Belgique",785053.0,50.859702,2.828440,...,Stijn Streuvelswijk,Ieper,None,België / Belgique / Belgien,8908,,None,orig,18,None
2,0669.833.203,Belgique,2100,Antwerpen,Boterlaarbaan,239,"Boterlaarbaan, 239, 2100 Antwerpen, Belgique",2010239.0,51.209175,4.470072,...,Boterlaarbaan,Deurne,239,België / Belgique / Belgien,2100,Eksterlaar,None,orig,239,None
3,2.243.108.073,Belgique,4460,Grâce-Hollogne,Rue de l'Aéropostale,8,"Rue de l'Aéropostale, 8, 4460 Grâce-Hollogne, ...",1445516.0,50.658359,5.483832,...,Rue de l'Aéropostale,Grâce-Hollogne,None,België / Belgique / Belgien,4460,Liège logistics,None,orig,8,None
4,2.045.477.107,Belgique,8210,Zedelgem,Koningin Astridstraat,109,"Koningin Astridstraat, 109, 8210 Zedelgem, Bel...",1763661.0,51.100407,3.167726,...,Koningin Astridstraat,Zedelgem,None,België / Belgique / Belgien,8210,,None,orig,109,"[{'index': 5.0, 'osm_addr_in': 'Koningin Astri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,0857.952.627,Belgique,8501,Kortrijk,Heulsestraat(Bis),3,"Heulsestraat, 3, 8501 Kortrijk, Belgique",2738948.0,50.834163,3.231044,...,Heulsestraat,Kortrijk,None,België / Belgique / Belgien,8501,,None,regex[init],3,None
93,2.249.583.121,Belgique,2018,Antwerpen,Desguinlei,90,"Desguinlei, 90, 2018 Antwerpen, Belgique",2323301.0,51.194730,4.407469,...,Desguinlei,Antwerpen,88-90,België / Belgique / Belgien,2018,Markgrave,None,orig,90,None
94,0834.315.707,Belgique,2018,Antwerpen,Diercxsensstraat,39,"Diercxsensstraat, 39, 2018 Antwerpen, Belgique",2338350.0,51.204519,4.398727,...,Diercxsensstraat,Antwerpen,39,België / Belgique / Belgien,2018,Brederode,None,orig,39,None
95,0857.635.002,Belgique,1700,Dilbeek,Ninoofsesteenweg,227,"Ninoofsesteenweg, 227, 1700 Dilbeek, Belgique",943444.0,50.843955,4.264415,...,Ninoofsesteenweg,Dilbeek,None,België / Belgique / Belgien,1700,,None,orig,227,"[{'index': 120.0, 'osm_addr_in': 'Ninoofsestee..."


### Batch blocs

In [22]:
chunk_size = 10
chunks = np.array_split(addresses, addresses.shape[0]//chunk_size)

res= [call_ws_batch(chunk, mode="long") for chunk in tqdm(chunks)]

## TODO : find a better way with dask? It seems that map_partitions does not support function returning dataframes. 
#50: 4:04
#100 : 2:30
#250 : 2:04
#1000 : 1:37

  0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
df_res = pd.concat(res, sort=False)
df_res

,addr_key,country,postcode,city,street,housenumber,json,osm_addr_in,place_id,lat,...,SIM_street_which,addr_out_street,addr_out_city,addr_out_number,addr_out_country,addr_out_postcode,addr_out_other,method,retry_on_26,extra_house_nbr
0,0836.380.520,Belgique,7911,Frasnes-lez-Anvaing,Rue Goderneau(MB),14,"{'match': [{'SIM_street_which': '', 'addr_out_...","Rue Goderneau, 14, 7911 Frasnes-lez-Anvaing, B...",695630.0,50.645420,...,None,Rue Goderneau,Ath,None,België / Belgique / Belgien,7906,,regex[init],None,14
1,2.231.595.460,Belgique,9981,Sint-Laureins,Vlamingstraat(STM),92,"{'match': [{'SIM_street_which': '', 'addr_out_...","Vlamingstraat, 92, 9981 Sint-Laureins, Belgique",2339251.0,51.250677,...,None,Vlamingstraat,Sint-Laureins,None,België / Belgique / Belgien,9981,,regex[init],None,92
2,0431.356.723,Belgique,1050,Ixelles,Rue de l'Amazone,21,"{'match': [{'SIM_street_which': '', 'addr_out_...","Rue de l'Amazone, 21, 1050 Ixelles, Belgique",203672.0,50.824780,...,None,Rue de l'Amazone - Amazonestraat,Ixelles - Elsene,21,België / Belgique / Belgien,1050,,orig,None,21
3,0832.380.556,Belgique,4000,Liège,Rue Henri-Maus,29,"{'match': [{'SIM_street_which': '', 'addr_out_...","Rue Henri-Maus, 29, 4000 Liège, Belgique",1602456.0,50.630432,...,None,Rue Henri Maus,Liège,None,België / Belgique / Belgien,4000,Glain,orig,None,29
4,2.168.200.715,Belgique,8300,Knokke-Heist,Zeedijk-Knokke,641,"{'match': [{'SIM_street_which': '', 'addr_out_...","Zeedijk-Knokke, 641, 8300 Knokke-Heist, Belgique",739194.0,51.352155,...,None,Zeedijk Knokke,Knokke-Heist,None,België / Belgique / Belgien,8300,Albertstrand,orig,None,641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2.244.614.642,Belgique,9240,Zele,Smeiersberg,5,"{'match': [{'SIM_street_which': '', 'addr_out_...","Smeiersberg, 5, 9240 Zele, Belgique",795193.0,51.058158,...,None,Smeiersberg,Zele,None,België / Belgique / Belgien,9240,,orig,NaN,5
6,2.252.606.947,Belgique,1080,Molenbeek-Saint-Jean,Rue Henri De Saegher,27,"{'match': [{'SIM_street_which': '', 'addr_out_...","Rue Henri De Saegher, 27, 1080 Molenbeek-Saint...",441880.0,50.852214,...,None,Rue Henri De Saegher - Henri De Saegherstraat,Molenbeek-Saint-Jean - Sint-Jans-Molenbeek,27,België / Belgique / Belgien,1080,Kaya market,orig,NaN,27
7,0650.498.331,Belgique,4700,Eupen,Stockbergerweg,5,"{'match': [{'SIM_street_which': '', 'addr_out_...","Stockbergerweg, 5, 4700 Eupen, Belgique",2413057.0,50.626646,...,None,Stockbergerweg,Eupen,None,België / Belgique / Belgien,4700,Unterstadt,orig,NaN,5
8,2.037.557.947,Belgique,1180,Uccle,Parvis Saint-Pierre,119,"{'match': [{'SIM_street_which': '', 'addr_out_...","Parvis Saint-Pierre, 119, 1180 Uccle, Belgique",485970.0,50.803207,...,None,Parvis Saint-Pierre - Sint-Pietersvoorplein,Uccle - Ukkel,None,België / Belgique / Belgien,1180,,orig,NaN,119


In [24]:
df_res.method.value_counts()

orig                             73
regex[init]                      19
libpostal+regex[lpost]+photon     3
nonum                             2
nostreet                          2
Name: method, dtype: int64

In [75]:
# df_res


## Comparing options

In [24]:
def call_ws_batch_chunks(addr_data, mode="geo", with_reject=False, check_result=True, structured_osm=False, chunk_size=50): 
    chunks = np.array_split(addr_data, addr_data.shape[0]//chunk_size)

    res= [call_ws_batch(chunk, mode=mode, 
                        check_result=check_result, 
                        structured_osm=structured_osm) for chunk in tqdm(chunks)]
    df_res = pd.concat(res, sort=False)
    return df_res

In [46]:
results = {}

In [48]:
results[("check", "struct")] = call_ws_batch_chunks(addresses, mode="short", check_result=True, structured_osm=True)

  0%|          | 0/20 [00:00<?, ?it/s]

In [49]:
results[("check", "unstruct")] = call_ws_batch_chunks(addresses, mode="short", check_result=True, structured_osm=False)

  0%|          | 0/20 [00:00<?, ?it/s]

In [50]:
results[("nocheck", "struct")] = call_ws_batch_chunks(addresses, mode="short", check_result=False, structured_osm=True)

  0%|          | 0/20 [00:00<?, ?it/s]

In [51]:
results[("nocheck", "unstruct")] = call_ws_batch_chunks(addresses, mode="short", check_result=False, structured_osm=False)

  0%|          | 0/20 [00:00<?, ?it/s]

In [103]:
for i in ["check","nocheck"]:
    for j in ["struct", "unstruct"]:
        print(f"{i:7}, {j:8}: {results[(i,j)].shape[0]}")

check  , struct  : 973
check  , unstruct: 976
nocheck, struct  : 973
nocheck, unstruct: 978


In [83]:
print("Only right:")
r2[r2.addr_key.isin(mg[mg._merge=="right_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1))

Only right:


,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
0,0828.907.065,50.888591,4.448425,30.0,photon,540543.0,De Kleetlaan,12,12B,1831,Machelen,België / Belgique / Belgien,Belgique,1831,Machelen (Brab.),De Kleetlaan,12B
1,2.247.616.593,50.097360,5.868338,26.0,nonum,779196.0,Buret,None,41,None,Houffalize,België / Belgique / Belgien,Belgique,6662,Houffalize,Buret,41
2,0835.605.807,49.798003,5.007217,26.0,nonum,2596140.0,Rue Saint-Jean-Baptiste,None,33,None,Bouillon,België / Belgique / Belgien,Belgique,6838,Bouillon,"Rue Saint-Jean-Baptiste,Corbion",33


In [120]:
vc_values = pd.DataFrame(columns=results.keys(), index=results.keys())
for k1 in results:
    for k2 in results:
        if k1>=k2:
            r1=results[k1]
            r2=results[k2]
            mg = r1[["addr_key", "place_id"]].merge(r2[["addr_key", "place_id"]], on="addr_key", how="outer", indicator=True)
            vc = mg._merge.value_counts()

            #print(f"{str(k1):25} vs {str(k2):25} : {vc['both']}/{vc['left_only']}/{vc['right_only']}")
            vc_values.loc[k1, k2]=f"{vc['both']} ({mg[mg.place_id_x == mg.place_id_y].shape[0]}) / {vc['left_only']} / {vc['right_only']}"
print("Common in both (agree on result) / results only for row / results only for columns")
vc_values.fillna("")

Common in both (agree on result) / results only for row / results only for columns


check                               nocheck  \
                             struct           unstruct             struct   
check   struct    973 (973) / 0 / 0                                         
        unstruct  973 (926) / 3 / 0  976 (976) / 0 / 0                      
nocheck struct    973 (953) / 0 / 0  973 (910) / 0 / 3  973 (973) / 0 / 0   
        unstruct  973 (909) / 5 / 0  976 (955) / 2 / 0  973 (925) / 5 / 0   

                                     
                           unstruct  
check   struct                       
        unstruct                     
nocheck struct                       
        unstruct  978 (978) / 0 / 0

In [125]:
for k1 in results:
    for k2 in results:
        if k1>k2:
            r1=results[k1]
            r2=results[k2]
            mg = r1[["addr_key", "place_id"]].merge(r2[["addr_key", "place_id"]], on="addr_key", how="outer", indicator=True)
            
            if (mg._merge=="left_only").any() :
                print("In ", k1, "not in ", k2)
    
                display(r1[r1.addr_key.isin(mg[mg._merge=="left_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            if (mg._merge=="right_only").any() :
                print("In ", k2, "not in ", k1)
        
                display(r2[r2.addr_key.isin(mg[mg._merge=="right_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            

In  ('check', 'unstruct') not in  ('check', 'struct')


,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
0,0828.907.065,50.888591,4.448425,30.0,photon,540543.0,De Kleetlaan,12,12B,1831,Machelen,België / Belgique / Belgien,Belgique,1831,Machelen (Brab.),De Kleetlaan,12B
1,2.247.616.593,50.097360,5.868338,26.0,nonum,779196.0,Buret,None,41,None,Houffalize,België / Belgique / Belgien,Belgique,6662,Houffalize,Buret,41
2,0835.605.807,49.798003,5.007217,26.0,nonum,2596140.0,Rue Saint-Jean-Baptiste,None,33,None,Bouillon,België / Belgique / Belgien,Belgique,6838,Bouillon,"Rue Saint-Jean-Baptiste,Corbion",33


In  ('check', 'unstruct') not in  ('nocheck', 'struct')


,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
0,0828.907.065,50.888591,4.448425,30.0,photon,540543.0,De Kleetlaan,12,12B,1831,Machelen,België / Belgique / Belgien,Belgique,1831,Machelen (Brab.),De Kleetlaan,12B
1,2.247.616.593,50.097360,5.868338,26.0,nonum,779196.0,Buret,None,41,None,Houffalize,België / Belgique / Belgien,Belgique,6662,Houffalize,Buret,41
2,0835.605.807,49.798003,5.007217,26.0,nonum,2596140.0,Rue Saint-Jean-Baptiste,None,33,None,Bouillon,België / Belgique / Belgien,Belgique,6838,Bouillon,"Rue Saint-Jean-Baptiste,Corbion",33


In  ('nocheck', 'unstruct') not in  ('check', 'struct')


,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
0,0828.907.065,50.888591,4.448425,30.0,photon,540543.0,De Kleetlaan,12,12B,1831,Machelen,België / Belgique / Belgien,Belgique,1831,Machelen (Brab.),De Kleetlaan,12B
1,2.247.616.593,50.097606,5.868829,19.0,nonum,109167.0,None,None,41,None,Buret,België / Belgique / Belgien,Belgique,6662,Houffalize,Buret,41
2,0835.605.807,49.798003,5.007217,26.0,nonum,2596140.0,Rue Saint-Jean-Baptiste,None,33,None,Bouillon,België / Belgique / Belgien,Belgique,6838,Bouillon,"Rue Saint-Jean-Baptiste,Corbion",33
3,0688.857.970,51.485611,4.734145,26.0,nostreet,514806.0,Amsterdamstraat,None,122,4836LA,None,Nederland,Pays-Bas,1101 EN,AMSTERDAM,BURGEMEESTER STRAMANWEG,122
4,0515.979.325,49.665874,5.495006,26.0,nostreet,627726.0,Rue du Duché,None,4,6730,Tintigny,België / Belgique / Belgien,Luxembourg (Grand-Duché),2530,LUXEMBOURG,RUE EHNRI M. SCHNADT,4


In  ('nocheck', 'unstruct') not in  ('check', 'unstruct')


,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
0,0688.857.970,51.485611,4.734145,26.0,nostreet,514806.0,Amsterdamstraat,None,122,4836LA,None,Nederland,Pays-Bas,1101 EN,AMSTERDAM,BURGEMEESTER STRAMANWEG,122
1,0515.979.325,49.665874,5.495006,26.0,nostreet,627726.0,Rue du Duché,None,4,6730,Tintigny,België / Belgique / Belgien,Luxembourg (Grand-Duché),2530,LUXEMBOURG,RUE EHNRI M. SCHNADT,4


In  ('nocheck', 'unstruct') not in  ('nocheck', 'struct')


,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
0,0828.907.065,50.888591,4.448425,30.0,photon,540543.0,De Kleetlaan,12,12B,1831,Machelen,België / Belgique / Belgien,Belgique,1831,Machelen (Brab.),De Kleetlaan,12B
1,2.247.616.593,50.097606,5.868829,19.0,nonum,109167.0,None,None,41,None,Buret,België / Belgique / Belgien,Belgique,6662,Houffalize,Buret,41
2,0835.605.807,49.798003,5.007217,26.0,nonum,2596140.0,Rue Saint-Jean-Baptiste,None,33,None,Bouillon,België / Belgique / Belgien,Belgique,6838,Bouillon,"Rue Saint-Jean-Baptiste,Corbion",33
3,0688.857.970,51.485611,4.734145,26.0,nostreet,514806.0,Amsterdamstraat,None,122,4836LA,None,Nederland,Pays-Bas,1101 EN,AMSTERDAM,BURGEMEESTER STRAMANWEG,122
4,0515.979.325,49.665874,5.495006,26.0,nostreet,627726.0,Rue du Duché,None,4,6730,Tintigny,België / Belgique / Belgien,Luxembourg (Grand-Duché),2530,LUXEMBOURG,RUE EHNRI M. SCHNADT,4


In [157]:
for k1 in results:
    for k2 in results:
        if k1>k2:
            r1=results[k1]
            r2=results[k2]
            mg = r1[["addr_key", "place_id"]].merge(r2[["addr_key", "place_id"]], on="addr_key", how="outer", indicator=True)
 
            mismatches = mg[mg.place_id_x != mg.place_id_y][["addr_key"]]
            mismatches = mismatches.merge(addresses.rename({addr_key_field:"addr_key"}, axis=1))
            mismatches = mismatches.merge(r1[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches = mismatches.merge(r2[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches.columns = pd.MultiIndex.from_arrays([["Input"]*6 + [f"x:{k1}"]*5 + [f"y:{k2}"]*5, mismatches.columns])

            print(k1, "vs", k2)
            display(mismatches)

('check', 'unstruct') vs ('check', 'struct')


Input                                         \
         addr_key CountryFR Zipcode       MunicipalityFR   
0   2.219.895.478  Belgique    5640               Mettet   
1   2.012.762.965  Belgique    9830   Sint-Martens-Latem   
2   2.118.442.089  Belgique    8587       Spiere-Helkijn   
3    0673.579.480  Belgique    1050              Ixelles   
4    0694.498.818  Belgique    6800   Libramont-Chevigny   
5   2.183.260.063  Belgique    5541             Hastière   
6    0659.733.523  Belgique    9600                Ronse   
7   2.064.858.596  Belgique    1000            Bruxelles   
8   2.188.381.069  Belgique    4000                Liège   
9    0642.730.215  Belgique    1050              Ixelles   
10   0704.239.794  Belgique    8908                Ieper   
11  2.276.311.074  Belgique    6686             Bertogne   
12  2.254.643.551  Belgique    1410             Waterloo   
13  2.000.798.016  Belgique    1030           Schaerbeek   
14   0895.368.594  Belgique    5590                Ciney   
15   0688.491.152  Belgique    5363               Hamois   
16  2.247.334.008  Belgique    1050              Ixelles   
17   0821.305.235  Belgique    4680               Oupeye   
18   0842.100.055  Belgique    1490  Court-Saint-Etienne   
19   0691.920.992  Belgique    1640   Sint-Genesius-Rode   
20   0856.302.538  Belgique    1050              Ixelles   
21   0719.723.370  Belgique    5660               Couvin   
22  2.250.653.980  Belgique    4040              Herstal   
23   0874.531.610  Belgique    1050              Ixelles   
24   0455.982.449  Belgique    2630           Aartselaar   
25   0874.906.049  Belgique    4040              Herstal   
26   0462.281.214  Belgique    1300                Wavre   
27  2.232.303.956  Belgique    4770                 Amel   
28   0691.540.120  Belgique    6900    Marche-en-Famenne   
29  2.012.615.980  Belgique    4100              Seraing   
30  2.214.745.768  Belgique    8400             Oostende   
31  2.259.181.369  Belgique    1620            Drogenbos   
32   0630.793.473  Belgique    1703              Dilbeek   
33   0642.891.947  Belgique    1050            Bruxelles   
34  2.265.756.979  Belgique    6900    Marche-en-Famenne   
35  2.181.655.605  Belgique    4651                Herve   
36  2.009.202.867  Belgique    8670             Koksijde   
37  2.089.681.688  Belgique    1315              Incourt   
38  2.266.123.797  Belgique    4880                Aubel   
39  2.052.338.965  Belgique    1040            Etterbeek   
40  2.240.458.983  Belgique    4350            Remicourt   
41  2.131.498.982  Belgique    1570           Galmaarden   
42   0445.381.735  Belgique    3050         Oud-Heverlee   
43  2.260.452.366  Belgique    1300                Wavre   
44  2.097.852.751  Belgique    5530                Yvoir   
45  2.244.569.607  Belgique    9700           Oudenaarde   
46  2.284.225.185  Belgique    9770              Kruisem   

                                                 \
                           StreetFR HouseNumber   
0                Rue Alvaux, Biesme          15   
1                      169 LATEMSES         NaN   
2                   Barrière de Fer          55   
3                     Avenue Louise         375   
4       Libramont,Rue de la Scierie          22   
5         Rue des Gaux, H.-par-delà          83   
6                 Rue de l'Ommegang        37 A   
7                         Rue Blaes        55 5   
8             Domaine Universitaire          ZN   
9                     Avenue Louise         350   
10                    Gezellestraat           9   
11                        Flamierge         959   
12              Square d'Argenteuil           5   
13                96 A.PR.ELISABETH         NaN   
14    Rue du Pays de Liège(Leignon)          31   
15     Chaussée d'Andenne, Emptinne          sn   
16                    Avenue Louise         164   
17                  Rue de Hermalle         102   
18                   Rue Albert Ier          51   


('nocheck', 'struct') vs ('check', 'struct')


Input                                       \
         addr_key CountryFR Zipcode     MunicipalityFR   
0    0862.153.222  Belgique    2320        Hoogstraten   
1   2.069.191.924  Belgique    3500            Hasselt   
2   2.281.590.646  Belgique    2200          Herentals   
3    0436.544.441  Belgique    1000          Bruxelles   
4   2.276.311.074  Belgique    6686           Bertogne   
5    0810.009.089  Belgique    1000          Bruxelles   
6   2.254.643.551  Belgique    1410           Waterloo   
7   2.263.958.224  Belgique    9111       Sint-Niklaas   
8    0414.600.863  Belgique    8380             Brugge   
9    0855.117.851  Belgique    4020              Liège   
10  2.258.119.022  Belgique    3150             Haacht   
11   0662.682.323  Belgique    8700              Tielt   
12   0446.309.173  Belgique    3010             Leuven   
13   0434.760.235  Belgique    1083          Ganshoren   
14   0561.798.759  Belgique    9880             Aalter   
15  2.265.756.979  Belgique    6900  Marche-en-Famenne   
16  2.181.655.605  Belgique    4651              Herve   
17  2.209.053.650  Belgique    1800          Vilvoorde   
18   0720.748.897  Belgique    2170          Antwerpen   
19  2.065.797.320  Belgique    9100       Sint-Niklaas   

                                           \
                     StreetFR HouseNumber   
0        Van Aertselaerstraat          53   
1                  Bergstraat          56   
2        Voortkapelseweg(NDW)         102   
3       Boulevard de Waterloo          11   
4                   Flamierge         959   
5        Chaussée de La Hulpe         120   
6         Square d'Argenteuil           5   
7                   Oude Baan          10   
8              Uitkerkestraat           7   
9       Bld Emile-de-Laveleye         132   
10           Lipsestraat(TIL)         165   
11               Bruggestraat          81   
12          Guido Gezellelaan           4   
13    Avenue Marie de Hongrie          61   
14            Oostmolen-Noord          14   
15  Marche, Rue du Luxembourg          2A   
16          Noblehaye,Battice        10/B   
17         Grimbergsesteenweg         153   
18            Vuurkruisenlaan          41   
19            Vlasbloemstraat           2   

                              x:('nocheck', 'struct')                    \
                                    addr_out_street_x addr_out_number_x   
0                                Van Aertselaerstraat              None   
1                                          Bergstraat              None   
2                                     Voortkapelseweg              None   
3                Boulevard de Waterloo - Waterloolaan              None   
4                                      Rue du Spitron              None   
5            Chaussée de La Hulpe - Terhulpsesteenweg              None   
6                                 Avenue d'Argenteuil              None   
7                                           Oude baan              None   
8                                      Uitkerkestraat              None   
9                         Boulevard Emile de Laveleye              None   
10                                        Lipsestraat              None   
11                                       Bruggestraat              None   
12                                  Guido Gezellelaan              None   
13  Avenue Marie de Hongrie - Maria van Hongarijelaan              None   
14                                    Oostmolen-Noord              None   
15                                               None              None   
16                     Route de Battice - Rue d'Aubel              None   
17                                 Grimbergsesteenweg              None   
18                                    Vuurkruisenlaan              None   
19                                    Vlasbloemstraat              None   

                                                             \
   extra_house_nbr_x addr_out_

('nocheck', 'struct') vs ('check', 'unstruct')


...

Input                                                              \
         addr_key CountryFR Zipcode      MunicipalityFR              StreetFR   
0   2.219.895.478  Belgique    5640              Mettet    Rue Alvaux, Biesme   
1    0862.153.222  Belgique    2320         Hoogstraten  Van Aertselaerstraat   
2   2.012.762.965  Belgique    9830  Sint-Martens-Latem          169 LATEMSES   
3   2.118.442.089  Belgique    8587      Spiere-Helkijn       Barrière de Fer   
4    0673.579.480  Belgique    1050             Ixelles         Avenue Louise   
..            ...       ...     ...                 ...                   ...   
59  2.065.797.320  Belgique    9100        Sint-Niklaas       Vlasbloemstraat   
60  2.260.452.366  Belgique    1300               Wavre     Avenue des Alizés   
61  2.097.852.751  Belgique    5530               Yvoir   Rue Grande, Godinne   
62  2.244.569.607  Belgique    9700          Oudenaarde        Ambachtsstraat   
63  2.284.225.185  Belgique    9770             Kruisem            Tramstatie   

                                  x:('nocheck', 'unstruct')                    \
   HouseNumber                            addr_out_street_x addr_out_number_x   
0           15                                   Rue Alvaux              None   
1           53                         Van Aertselaerstraat              None   
2          NaN                                         None              None   
3           55                             Jacquetbosstraat              None   
4          375  Avenue de la Toison d'Or - Gulden-Vlieslaan              None   
..         ...                                          ...               ...   
59           2                              Vlasbloemstraat              None   
60           3                                         None              None   
61          45                                   Rue Grande                45   
62          11                                         None              None   
63           6                                         None              None   

                                                                      \
   extra_house_nbr_x addr_out_postcode_x             addr_out_city_x   
0                 15                None                      Mettet   
1                 53                2320                 Hoogstraten   
2                169                9830          Sint-Martens-Latem   
3                 55                8587              Spiere-Helkijn   
4                375                None  Saint-Gilles - Sint-Gillis   
..               ...                 ...                         ...   
59                 2                9100                Sint-Niklaas   
60                 3                None                    Nivelles   
61                45                5530                       Yvoir   
62                11                9700                  Oudenaarde   
63                 6                9770                     Kruisem   

                         y:('check', 'struct')                    \
                             addr_out_street_y addr_out_number_y   
0                                Rue de Biesme              None   
1                         Van Aertselaerstraat                53   
2                                         None              None   
3                                         None              None   
4   Avenue Louis Lepoutre - Louis Lepoutrelaan              None   
..                                         ...               ...   
59                             Vlasbloemstraat                 2   
60                          Avenues des Alizés              None   
61                                        None              None   
62                                        None              None   
63                                        None              None   

                                                              
   extra_house_nbr_

('nocheck', 'unstruct') vs ('check', 'unstruct')


Input                                                            \
         addr_key CountryFR Zipcode MunicipalityFR                 StreetFR   
0    0862.153.222  Belgique    2320    Hoogstraten     Van Aertselaerstraat   
1   2.069.191.924  Belgique    3500        Hasselt               Bergstraat   
2   2.116.920.674  Belgique    2400            Mol             Ginderbuiten   
3   2.281.590.646  Belgique    2200      Herentals     Voortkapelseweg(NDW)   
4    0436.544.441  Belgique    1000      Bruxelles    Boulevard de Waterloo   
5   2.276.311.074  Belgique    6686       Bertogne                Flamierge   
6    0810.009.089  Belgique    1000      Bruxelles     Chaussée de La Hulpe   
7   2.263.958.224  Belgique    9111   Sint-Niklaas                Oude Baan   
8    0414.600.863  Belgique    8380         Brugge           Uitkerkestraat   
9    0855.117.851  Belgique    4020          Liège    Bld Emile-de-Laveleye   
10   0821.305.235  Belgique    4680         Oupeye          Rue de Hermalle   
11  2.258.119.022  Belgique    3150         Haacht         Lipsestraat(TIL)   
12  2.247.616.593  Belgique    6662     Houffalize                    Buret   
13   0662.682.323  Belgique    8700          Tielt             Bruggestraat   
14   0446.309.173  Belgique    3010         Leuven        Guido Gezellelaan   
15   0434.760.235  Belgique    1083      Ganshoren  Avenue Marie de Hongrie   
16   0561.798.759  Belgique    9880         Aalter          Oostmolen-Noord   
17  2.181.655.605  Belgique    4651          Herve        Noblehaye,Battice   
18  2.209.053.650  Belgique    1800      Vilvoorde       Grimbergsesteenweg   
19   0720.748.897  Belgique    2170      Antwerpen          Vuurkruisenlaan   
20  2.065.797.320  Belgique    9100   Sint-Niklaas          Vlasbloemstraat   

                                        x:('nocheck', 'unstruct')  \
   HouseNumber                                  addr_out_street_x   
0           53                               Van Aertselaerstraat   
1           56                                         Bergstraat   
2          NaN                                               None   
3          102                                    Voortkapelseweg   
4           11               Boulevard de Waterloo - Waterloolaan   
5          959                                               None   
6          120           Chaussée de La Hulpe - Terhulpsesteenweg   
7           10                                          Oude baan   
8            7                                     Uitkerkestraat   
9          132                        Boulevard Emile de Laveleye   
10         102                                   Rue de la Vallée   
11         165                                        Lipsestraat   
12          41                                               None   
13          81                                       Bruggestraat   
14           4                                  Guido Gezellelaan   
15          61  Avenue Marie de Hongrie - Maria van Hongarijelaan   
16          14                                    Oostmolen-Noord   
17        10/B                                               None   
18         153                                 Grimbergsesteenweg   
19          41                                    Vuurkruisenlaan   
20           2                                    Vlasbloemstraat   

                                                                            \
   addr_out_number_x extra_house_nbr_x addr_out_postcode_x addr_out_city_x   
0               None                53                2320     Hoogstraten   
1               None                56                3500         Hasselt   
2               None              None                2400    Ginderbuiten   
3               None               102                2200       Herentals   
4               None                11                1000             BXL   
5               None               959                6686  

('nocheck', 'unstruct') vs ('nocheck', 'struct')


Input                                         \
         addr_key CountryFR Zipcode       MunicipalityFR   
0   2.219.895.478  Belgique    5640               Mettet   
1   2.012.762.965  Belgique    9830   Sint-Martens-Latem   
2   2.118.442.089  Belgique    8587       Spiere-Helkijn   
3    0673.579.480  Belgique    1050              Ixelles   
4    0694.498.818  Belgique    6800   Libramont-Chevigny   
5   2.183.260.063  Belgique    5541             Hastière   
6    0659.733.523  Belgique    9600                Ronse   
7   2.064.858.596  Belgique    1000            Bruxelles   
8   2.188.381.069  Belgique    4000                Liège   
9    0642.730.215  Belgique    1050              Ixelles   
10  2.116.920.674  Belgique    2400                  Mol   
11   0704.239.794  Belgique    8908                Ieper   
12  2.276.311.074  Belgique    6686             Bertogne   
13  2.254.643.551  Belgique    1410             Waterloo   
14  2.000.798.016  Belgique    1030           Schaerbeek   
15   0895.368.594  Belgique    5590                Ciney   
16   0688.491.152  Belgique    5363               Hamois   
17  2.247.334.008  Belgique    1050              Ixelles   
18   0821.305.235  Belgique    4680               Oupeye   
19   0842.100.055  Belgique    1490  Court-Saint-Etienne   
20   0691.920.992  Belgique    1640   Sint-Genesius-Rode   
21   0856.302.538  Belgique    1050              Ixelles   
22   0719.723.370  Belgique    5660               Couvin   
23  2.250.653.980  Belgique    4040              Herstal   
24   0874.531.610  Belgique    1050              Ixelles   
25   0455.982.449  Belgique    2630           Aartselaar   
26   0874.906.049  Belgique    4040              Herstal   
27   0462.281.214  Belgique    1300                Wavre   
28  2.232.303.956  Belgique    4770                 Amel   
29   0691.540.120  Belgique    6900    Marche-en-Famenne   
30  2.012.615.980  Belgique    4100              Seraing   
31  2.214.745.768  Belgique    8400             Oostende   
32  2.259.181.369  Belgique    1620            Drogenbos   
33   0630.793.473  Belgique    1703              Dilbeek   
34   0642.891.947  Belgique    1050            Bruxelles   
35  2.265.756.979  Belgique    6900    Marche-en-Famenne   
36  2.181.655.605  Belgique    4651                Herve   
37  2.009.202.867  Belgique    8670             Koksijde   
38  2.089.681.688  Belgique    1315              Incourt   
39  2.266.123.797  Belgique    4880                Aubel   
40  2.052.338.965  Belgique    1040            Etterbeek   
41  2.240.458.983  Belgique    4350            Remicourt   
42  2.131.498.982  Belgique    1570           Galmaarden   
43   0445.381.735  Belgique    3050         Oud-Heverlee   
44  2.260.452.366  Belgique    1300                Wavre   
45  2.097.852.751  Belgique    5530                Yvoir   
46  2.244.569.607  Belgique    9700           Oudenaarde   
47  2.284.225.185  Belgique    9770              Kruisem   

                                                 \
                           StreetFR HouseNumber   
0                Rue Alvaux, Biesme          15   
1                      169 LATEMSES         NaN   
2                   Barrière de Fer          55   
3                     Avenue Louise         375   
4       Libramont,Rue de la Scierie          22   
5         Rue des Gaux, H.-par-delà          83   
6                 Rue de l'Ommegang        37 A   
7                         Rue Blaes        55 5   
8             Domaine Universitaire          ZN   
9                     Avenue Louise         350   
10                     Ginderbuiten         NaN   
11                    Gezellestraat           9   
12                        Flamierge         959   
13              Square d'Argenteuil           5   
14                96 A.PR.ELISABETH         NaN   
15    Rue du Pays de Liège(Leignon)          31   
16     Chaussée d'Andenne, Emptinne          sn   
17                    Avenue Louise       

In [155]:
mismatches

Input                                         \
         addr_key CountryFR Zipcode       MunicipalityFR   
0   2.219.895.478  Belgique    5640               Mettet   
1   2.012.762.965  Belgique    9830   Sint-Martens-Latem   
2   2.118.442.089  Belgique    8587       Spiere-Helkijn   
3    0673.579.480  Belgique    1050              Ixelles   
4    0694.498.818  Belgique    6800   Libramont-Chevigny   
5   2.183.260.063  Belgique    5541             Hastière   
6    0659.733.523  Belgique    9600                Ronse   
7   2.064.858.596  Belgique    1000            Bruxelles   
8   2.188.381.069  Belgique    4000                Liège   
9    0642.730.215  Belgique    1050              Ixelles   
10  2.116.920.674  Belgique    2400                  Mol   
11   0704.239.794  Belgique    8908                Ieper   
12  2.276.311.074  Belgique    6686             Bertogne   
13  2.254.643.551  Belgique    1410             Waterloo   
14  2.000.798.016  Belgique    1030           Schaerbeek   
15   0895.368.594  Belgique    5590                Ciney   
16   0688.491.152  Belgique    5363               Hamois   
17  2.247.334.008  Belgique    1050              Ixelles   
18   0821.305.235  Belgique    4680               Oupeye   
19   0842.100.055  Belgique    1490  Court-Saint-Etienne   
20   0691.920.992  Belgique    1640   Sint-Genesius-Rode   
21   0856.302.538  Belgique    1050              Ixelles   
22   0719.723.370  Belgique    5660               Couvin   
23  2.250.653.980  Belgique    4040              Herstal   
24   0874.531.610  Belgique    1050              Ixelles   
25   0455.982.449  Belgique    2630           Aartselaar   
26   0874.906.049  Belgique    4040              Herstal   
27   0462.281.214  Belgique    1300                Wavre   
28  2.232.303.956  Belgique    4770                 Amel   
29   0691.540.120  Belgique    6900    Marche-en-Famenne   
30  2.012.615.980  Belgique    4100              Seraing   
31  2.214.745.768  Belgique    8400             Oostende   
32  2.259.181.369  Belgique    1620            Drogenbos   
33   0630.793.473  Belgique    1703              Dilbeek   
34   0642.891.947  Belgique    1050            Bruxelles   
35  2.265.756.979  Belgique    6900    Marche-en-Famenne   
36  2.181.655.605  Belgique    4651                Herve   
37  2.009.202.867  Belgique    8670             Koksijde   
38  2.089.681.688  Belgique    1315              Incourt   
39  2.266.123.797  Belgique    4880                Aubel   
40  2.052.338.965  Belgique    1040            Etterbeek   
41  2.240.458.983  Belgique    4350            Remicourt   
42  2.131.498.982  Belgique    1570           Galmaarden   
43   0445.381.735  Belgique    3050         Oud-Heverlee   
44  2.260.452.366  Belgique    1300                Wavre   
45  2.097.852.751  Belgique    5530                Yvoir   
46  2.244.569.607  Belgique    9700           Oudenaarde   
47  2.284.225.185  Belgique    9770              Kruisem   

                                                 \
                           StreetFR HouseNumber   
0                Rue Alvaux, Biesme          15   
1                      169 LATEMSES         NaN   
2                   Barrière de Fer          55   
3                     Avenue Louise         375   
4       Libramont,Rue de la Scierie          22   
5         Rue des Gaux, H.-par-delà          83   
6                 Rue de l'Ommegang        37 A   
7                         Rue Blaes        55 5   
8             Domaine Universitaire          ZN   
9                     Avenue Louise         350   
10                     Ginderbuiten         NaN   
11                    Gezellestraat           9   
12                        Flamierge         959   
13              Square d'Argenteuil           5   
14                96 A.PR.ELISABETH         NaN   
15    Rue du Pays de Liège(Leignon)          31   
16     Chaussée d'Andenne, Emptinne          sn   
17                    Avenue Louise       

In [156]:
k1, k2



(('nocheck', 'unstruct'), ('nocheck', 'unstruct'))

MultiIndex([('Input',            'addr_key'),
            ('Input',           'CountryFR'),
            ('Input',             'Zipcode'),
            ('Input',      'MunicipalityFR'),
            ('Input',            'StreetFR'),
            ('Input',         'HouseNumber'),
            (    'X',   'addr_out_street_x'),
            (    'X',   'addr_out_number_x'),
            (    'X',   'extra_house_nbr_x'),
            (    'X', 'addr_out_postcode_x'),
            (    'X',     'addr_out_city_x'),
            (    'Y',   'addr_out_street_y'),
            (    'Y',   'addr_out_number_y'),
            (    'Y',   'extra_house_nbr_y'),
            (    'Y', 'addr_out_postcode_y'),
            (    'Y',     'addr_out_city_y')],
           )